### Statistiques concernant les cararactéristiques des incidents ayant fait des victimes : comment expliquer le nombre de victimes en fonction de l'incident ?

In [20]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)
import plotly.express as px

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
url="https://drive.google.com/file/d/1LO4i_-TkSmxnR0dgO5_lVLZ35FtGTNzA/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
total_db = pd.read_csv(url)
total_db.sample(5)

,Unnamed: 0,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,geometry,fips,unemp_rate_year_2013,unemp_rate_year_2014,unemp_rate_year_2015,unemp_rate_year_2016,unemp_rate_year_2017,unemp_rate_year_2018,med_h_income_year_2013,med_h_income_year_2014,med_h_income_year_2015,med_h_income_year_2016,med_h_income_year_2017,med_h_income_year_2018,resident_pop_year_2013,resident_pop_year_2014,resident_pop_year_2015,resident_pop_year_2016,resident_pop_year_2017,resident_pop_year_2018,snap_beneficiaries_year_2013,snap_beneficiaries_year_2014,snap_beneficiaries_year_2015,snap_beneficiaries_year_2016,snap_beneficiaries_year_2017,snap_beneficiaries_year_2018,bachelors_deg_year_2013,bachelors_deg_year_2014,bachelors_deg_year_2015,bachelors_deg_year_2016,bachelors_deg_year_2017,bachelors_deg_year_2018,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop
2692,2692,Martin,Texas,TX,27326,30054,Martin,"POLYGON ((-102.211033 32.177037, -102.211111 3...",48317.0,3.6,2.7,3.3,4.4,2.3,2.5,50004.0,61896.0,60815.0,58760.0,59907.0,56484.0,5.268,5.457,5.639,5.627,5.526,5.668,634.0,515.0,576.0,682.0,656.0,568.0,10.9,17.3,17.5,19.3,20.6,20.4,2.45,1633278,1465992,1574266,1721726,1549434,1571632,0.151567,0.827920
930,930,Grant,Kansas,KS,27298,28268,Grant,"POLYGON ((-101.527063 37.736314, -101.526365 3...",20067.0,3.0,3.0,3.3,3.4,2.7,2.8,54952.0,60279.0,59218.0,58620.0,59534.0,61614.0,7.843,7.784,7.719,7.660,7.476,7.271,522.0,528.0,538.0,439.0,397.0,393.0,17.8,18.5,18.2,15.2,16.2,15.4,0.90,228105,172167,185859,196548,191451,172047,0.184768,0.759412
1836,1836,Socorro,New Mexico,NM,27313,29186,Socorro,"POLYGON ((-107.726436 34.504469, -107.725319 3...",35053.0,7.0,6.6,6.9,6.9,5.3,5.6,32090.0,32285.0,33673.0,33239.0,34880.0,33989.0,17.507,17.255,17.148,16.989,16.825,16.649,5106.0,5669.0,5345.0,5182.0,5429.0,5167.0,19.4,19.2,17.5,19.6,19.5,21.2,1.25,156333,139780,147912,161216,159779,156853,0.484754,0.383659
2816,2816,Wasatch,Utah,UT,27328,30180,Wasatch,"POLYGON ((-111.620725 40.451322, -111.619412 4...",49051.0,3.5,2.9,3.0,2.7,2.6,3.0,64423.0,65207.0,75112.0,73077.0,83344.0,85380.0,26.578,27.815,29.131,30.433,32.045,33.278,1082.0,1024.0,970.0,846.0,809.0,684.0,33.4,34.4,33.5,36.9,38.7,40.6,2.00,324604,263812,254814,294907,270477,295858,0.252723,0.504538
1010,1010,Bourbon,Kentucky,KY,152,28349,Bourbon,"POLYGON ((-84.442661 38.283235999999995, -84.4...",21017.0,6.0,4.1,4.6,4.6,3.6,4.4,40045.0,42812.0,46370.0,46231.0,46281.0,48614.0,20.023,20.049,20.159,20.123,20.088,20.102,4031.0,3741.0,3236.0,3209.0,2962.0,2711.0,15.8,15.7,16.0,17.8,19.0,18.9,1.25,1578331,2492184,3218371,3676847,4641480,4912441,0.317484,0.633489


In [6]:
gun_violence_db = pd.read_csv('df_gun_violence.csv')
gun_violence_db.sample(5)

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict
7482,6310,110629,2014-02-26,Florida,Baker,NaN,0,0,http://www.gunviolencearchive.org/incident/110629,http://www.nwfdailynews.com/local/carjacking-s...,False,1.0,NaN,NaN,NaN,30.8415,NaN,-86.6755,NaN,NaN,{1: '21'},"{0: 'Adult 18+', 1: 'Adult 18+'}","{0: 'Female', 1: 'Male'}",1::Waylon Dwight Ganey,NaN,"{0: 'Unharmed', 1: 'Unharmed'}","{0: 'Victim', 1: 'Subject-Suspect'}",NaN,3.0,2.0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
20328,18165,144725,2014-06-05,California,Hayward,21000 block of Foothill Boulevard,1,0,http://www.gunviolencearchive.org/incident/144725,http://www.sfgate.com/crime/article/Hayward-ki...,False,15.0,NaN,NaN,NaN,37.6868,NaN,-122.0940,NaN,NaN,"{0: '24', 1: '34'}",{0: 'Adult 18+'},"{0: 'Male', 1: 'Male'}",0::Carlos Cetina||1::Pedro Cruz,NaN,{0: 'Killed'},"{0: 'Victim', 1: 'Subject-Suspect'}",['http://www.insidebayarea.com/news/ci_2593561...,20.0,10.0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
197784,196706,888432,2017-07-11,Mississippi,Lucedale,S Pine St and Estes St,1,0,http://www.gunviolencearchive.org/incident/888432,http://www.wmbfnews.com/story/35867942/suspect...,False,4.0,{0: 'Unknown'},{0: 'Unknown'},NaN,30.9175,NaN,-88.5963,1.0,NaN,"{0: '33', 1: '31'}","{0: 'Adult 18+', 1: 'Adult 18+'}","{0: 'Male', 1: 'Male'}",0::Demetrius Pate||1::Demetrius Spivery,NaN,"{0: 'Killed', 1: 'Unharmed, Arrested'}","{0: 'Victim', 1: 'Subject-Suspect'}",['http://www.wlox.com/story/35867942/lucedale-...,107.0,43.0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0
148575,147179,666844,2016-09-28,Missouri,Cape Girardeau,1213 South Ellis Street,1,0,http://www.gunviolencearchive.org/incident/666844,http://www.semissourian.com/story/2345090.html,False,8.0,{0: 'Unknown'},{0: 'Unknown'},"['Shot - Dead (murder, accidental, suicide)', ...",37.2871,NaN,-89.5310,1.0,NaN,"{0: '30', 1: '30', 2: '22'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+'}","{0: 'Male', 1: 'Male', 2: 'Male'}",0::Brent Shonta Johnson||1::Richard L. Jenkins...,NaN,"{0: 'Killed', 1: 'Unharmed, Arrested', 2: 'Unh...","{0: 'Victim', 1: 'Subject-Suspect', 2: 'Subjec...",['http://www.kfvs12.com/story/33266484/police-...,147.0,27.0,1,0,0,0,1,1,1,1,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0
69789,68054,339138,2015-05-12,New York,Brooklyn,Throop Avenue and Quincy Street,0,2,http://www.gunviolencearchive.org/incident/339138,http://www.nydailynews.com/new-york/brooklyn/n...,False,8.0,NaN,NaN,"['Shot - Wounded/Injured', 'Institution/Group/...",40.6879,NaN,-73.9419,NaN,"Bedford Stuyvevsant, bodega ar, gunfire exchan...","{0: '19', 1: '24'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Female', 2: 'Male', 3: 'Male'}",1::Elena Portillo,"{2: 'Armed Robbery', 3: 'Armed Robbery'}","{0: 'Injured', 1: 'Injured', 2: 'Unharmed', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Subject-Suspect...",['http://www.nydailynews.com/new-york/nyc-crim...,56.0,25.0,0,1,0,0,0,1,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1


In [7]:
from shapely import wkt
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None
total_db = total_db.dropna(subset=['geometry'])
total_db['geometry'] = total_db['geometry'].apply(wkt_loads)

In [8]:
total_db = total_db.loc[total_db['Etat'] != 'Alaska']
total_db = total_db.loc[total_db['Etat'] != 'Hawaii']

gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Alaska']
gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Hawaii']


In [9]:
total_db_geo = (
    gpd.GeoDataFrame(total_db)
    .set_geometry('geometry')
    .set_crs('EPSG:4269')
)

In [10]:
#changement de valeur pour Richmond, Virginia car valeur aberrante de 8.755 alors qu'il y avait 204 214	habitants en 2010
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2013'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2014'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2015'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2016'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2017'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2018'] = 204.214

In [11]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(gun_violence_db['longitude'], gun_violence_db['latitude'])]
gun_violence_geo = gpd.GeoDataFrame(gun_violence_db, geometry=geometry, crs='EPSG:4326')

gun_violence_geo = gun_violence_geo.set_crs('EPSG:4326')
gun_violence_geo = gun_violence_geo[gun_violence_geo.geometry.x <= -50]

In [12]:
total_db_geo=total_db_geo.to_crs('EPSG:4326')

In [13]:
merge_geo = gpd.sjoin(gun_violence_geo, total_db_geo, predicate='within')

In [14]:
geom_county=total_db_geo[['id_county','geometry']]
geom_county.head()

,id_county,geometry
0,27336,"POLYGON ((-86.92119 32.65754, -86.92035 32.658..."
1,27337,"POLYGON ((-88.02858 30.22676, -88.02399 30.230..."
2,27338,"POLYGON ((-85.74803 31.61918, -85.74543 31.618..."
3,27339,"POLYGON ((-87.42194 33.00338, -87.31854 33.006..."
4,27340,"POLYGON ((-86.96336 33.85822, -86.95967 33.857..."


In [15]:
merge_geo2=pd.merge(merge_geo, geom_county, on='id_county')
merge_geo2.head()

,Unnamed: 0_left,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict,geometry_x,index_right,Unnamed: 0_right,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,fips,unemp_rate_year_2013,unemp_rate_year_2014,unemp_rate_year_2015,unemp_rate_year_2016,unemp_rate_year_2017,unemp_rate_year_2018,med_h_income_year_2013,med_h_income_year_2014,med_h_income_year_2015,med_h_income_year_2016,med_h_income_year_2017,med_h_income_year_2018,resident_pop_year_2013,resident_pop_year_2014,resident_pop_year_2015,resident_pop_year_2016,resident_pop_year_2017,resident_pop_year_2018,snap_beneficiaries_year_2013,snap_beneficiaries_year_2014,snap_beneficiaries_year_2015,snap_beneficiaries_year_2016,snap_beneficiaries_year_2017,snap_beneficiaries_year_2018,bachelors_deg_year_2013,bachelors_deg_year_2014,bachelors_deg_year_2015,bachelors_deg_year_2016,bachelors_deg_year_2017,bachelors_deg_year_2018,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop,geometry_y
0,100178,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Mass Shooting (4+ ...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/01/01/4-...,NaN,NaN,0,0,0,0,0,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,POINT (-79.85590 40.34670),2258,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
1,104811,481268,2013-10-21,Pennsylvania,Pittsburgh (Homewood),7200 block of Kelly Street,2,5,http://www.gunviolencearchive.org/incident/481268,http://www.wtae.com/news/local/allegheny/two-p...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Shot - Dead (murde...",40.4555,Diverse Banquet Hall,-79.8970,NaN,NaN,"{0: '21', 1: '27'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Female', 3: 'Fema...",0::Jasmine Morris||1::Demetrius Broadnax,NaN,"{0: 'Killed', 1: 'Killed', 2: 'Injured', 3: 'I...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/10/20/ho...,NaN,NaN,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,POINT (-79.89700 40.45550),2258,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
2,678,94128,2014-01-08,Pennsylvania,Homestead,Steak ‘n Shake at The Waterfront,1,0,http://www.gunviolencearchive.org/incident/94128,http://pittsburg

In [16]:
merge_geo = merge_geo2.rename(columns={'geometry_x': 'point_geometry', 'geometry_y': 'polygon_geometry'})
merge_geo.head()

,Unnamed: 0_left,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict,point_geometry,index_right,Unnamed: 0_right,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,fips,unemp_rate_year_2013,unemp_rate_year_2014,unemp_rate_year_2015,unemp_rate_year_2016,unemp_rate_year_2017,unemp_rate_year_2018,med_h_income_year_2013,med_h_income_year_2014,med_h_income_year_2015,med_h_income_year_2016,med_h_income_year_2017,med_h_income_year_2018,resident_pop_year_2013,resident_pop_year_2014,resident_pop_year_2015,resident_pop_year_2016,resident_pop_year_2017,resident_pop_year_2018,snap_beneficiaries_year_2013,snap_beneficiaries_year_2014,snap_beneficiaries_year_2015,snap_beneficiaries_year_2016,snap_beneficiaries_year_2017,snap_beneficiaries_year_2018,bachelors_deg_year_2013,bachelors_deg_year_2014,bachelors_deg_year_2015,bachelors_deg_year_2016,bachelors_deg_year_2017,bachelors_deg_year_2018,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop,polygon_geometry
0,100178,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Mass Shooting (4+ ...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/01/01/4-...,NaN,NaN,0,0,0,0,0,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,POINT (-79.85590 40.34670),2258,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
1,104811,481268,2013-10-21,Pennsylvania,Pittsburgh (Homewood),7200 block of Kelly Street,2,5,http://www.gunviolencearchive.org/incident/481268,http://www.wtae.com/news/local/allegheny/two-p...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Shot - Dead (murde...",40.4555,Diverse Banquet Hall,-79.8970,NaN,NaN,"{0: '21', 1: '27'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Female', 3: 'Fema...",0::Jasmine Morris||1::Demetrius Broadnax,NaN,"{0: 'Killed', 1: 'Killed', 2: 'Injured', 3: 'I...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/10/20/ho...,NaN,NaN,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,POINT (-79.89700 40.45550),2258,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
2,678,94128,2014-01-08,Pennsylvania,Homestead,Steak ‘n Shake at The Waterfront,1,0,http://www.gunviolencearchive.org/incident/94128,http:/

In [17]:
base_totale=merge_geo.copy()

## Après cet import compliqué, on peut passer aux statistiques

In [73]:
print("I. Les dates. \nNous allons commencer cette analyse des conséquences des incidents en termes de victimes (blessés et morts) par l'étude des effets potentiels de la date sur le nombre et la gravité des incidents.")

I. Les dates. 
Nous allons commencer cette analyse des conséquences des incidents en termes de victimes (blessés et morts) par l'étude des effets potentiels de la date sur le nombre et la gravité des incidents.


In [23]:
base_totale['date']=pd.to_datetime(base_totale['date'])

In [43]:
grouped_data = base_totale.groupby(base_totale['date'].dt.year).agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()

fig1 = px.bar(
    grouped_data,
    x=grouped_data['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "année", "value": "Nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes d'incidents par arme à feu par an",
    color_discrete_map={"n_injured": "blue", "n_killed": "red"},
)

fig1.update_layout(barmode='group')  # Display bars side by side

fig1.show()


In [49]:
print("Ce graphique nous montre que, non seulement le nombre d'incidents par arme à feu augmente sur nos 4 années complètes considérées, mais que le nombre de victimes (blessés et tués) augmente également. Ainsi, les conséquences de ces incidents touchent de plus en plus d'individus au fur et à mesure des années.")

Ce graphique nous montre que, non seulement le nombre d'incidents par arme à feu augmente sur nos 4 années complètes considérées, mais que le nombre de victimes (blessés et tués) augmente également. Ainsi, les conséquences de ces incidents touchent de plus en plus d'individus au fur et à mesure des années.


In [57]:
grouped_data2 = base_totale.groupby(base_totale['date'].dt.year).agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()

grouped_data2=grouped_data2.loc[grouped_data2['date'].isin([2014,2015,2016,2017])]

fig2 = px.bar(
    grouped_data2,
    x=grouped_data2['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "année", "value": "Moyenne de victimes par incident", "variable": "Type de victime"},
    title="Nombre moyen de victimes par incident par arme à feu par an",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig2.update_layout(barmode='group')  # Display bars side by side

fig2.show()

In [47]:
print("Ce second graphique nous montre que l'augmentation des victimes repérée précédemment est liée quasiment exclusivement à l'augementation du nombre d'incident, car le nombre de victime par incident reste relativement stable sur les 4 années observées, mis à part une légèrement augementation du nombre de blessés moyen par incident entre 2014 et 2015.")

Ce second graphique nous montre que l'augmentation des victimes repérée précédemment est liée quasiment exclusivement à l'augementation du nombre d'incident, car le nombre de victime par incident reste relativement stable sur les 4 années observées, mis à part une légèrement augementation du nombre de blessés moyen par incident entre 2014 et 2015.


In [66]:
grouped_data3 = base_totale.loc[base_totale['date'].dt.year.isin([2014,2015,2016,2017])].groupby(
    base_totale['date'].dt.month).agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig3 = px.bar(
    grouped_data3,
    x=grouped_data3['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "Mois", "value": "Nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes d'incidents par arme à feu par mois (cumulé de 2014 à 2017)",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig3.update_layout(barmode='group')  # Display bars side by side

fig3.show()



In [68]:
grouped_data4 = base_totale.loc[base_totale['date'].dt.year.isin([2014,2015,2016,2017])].groupby(
    base_totale['date'].dt.month).agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig4 = px.bar(
    grouped_data4,
    x=grouped_data4['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "année", "value": "Nombre moyen de victimes par incident", "variable": "Type de victime"},
    title="Moyenne de victimes d'incident par arme à feu par mois (entre 2014 et 2017)",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig4.update_layout(barmode='group')  # Display bars side by side

fig4.show()

In [70]:
print("On retrouve ici la même variation que pour le nombre d'incidents : il y a plus de blessés et de morts par arme à feu pendant l'été aux Etats-Unis que pendant l'hiver. \nDe plus, même si le nombre de morts par incident reste stable autour de 0,25, le nombre moyen de victimes par incident varie de 0.46 en Février à 0.55 en Juillet. \nIl semble donc exister un effet de la saisonnalité sur le nombre d'incidents mais également sur leur gravité et leurs conséquences en termes de victimes.")

On retrouve ici la même variation que pour le nombre d'incidents : il y a plus de blessés et de morts par arme à feu pendant l'été aux Etats-Unis que pendant l'hiver. 
De plus, même si le nombre de morts par incident reste stable autour de 0,25, le nombre moyen de victimes par incident varie de 0.46 en Février à 0.55 en Juillet. 
Il semble donc exister un effet de la saisonnalité sur le nombre d'incidents mais également sur leur gravité et leurs conséquences en termes de victimes.


In [75]:
print("II. Les Etats. \nCette analyse des incidents en fonction des Etats sera complétée par une visualisation cartographique dans une partie suivante. \nIntéressons nous d'abord aux conséquences des incidents par arme à feu dans chaque Etat.")

II. Les Etats. 
Cette analyse des incidents en fonction des Etats sera complétée par une visualisation cartographique dans une partie suivante. 
Intéressons nous d'abord aux conséquences des incidents par arme à feu dans chaque Etat.


In [78]:
grouped_data5 = base_totale.groupby('state').agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig5 = px.bar(
    grouped_data5,
    x=grouped_data5['state'],
    y=['n_injured', 'n_killed'],
    labels={"state": "Etat", "value": "Nombre de victimes par incident", "variable": "Type de victime"},
    title="Nombre de victimes d'incident par arme à feu par Etat",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig5.update_layout(barmode='group')  # Display bars side by side

fig5.show()

In [80]:
grouped_data6 = base_totale.groupby('state').agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig6 = px.bar(
    grouped_data6,
    x=grouped_data6['state'],
    y=['n_injured', 'n_killed'],
    labels={"state": "Etat", "value": "Moyenne du nombre de victimes par incident", "variable": "Type de victime"},
    title="Nombre moyen de victimes d'incident par arme à feu par Etat",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig6.update_layout(barmode='group')  # Display bars side by side

fig6.show()

In [85]:
grouped_data7 = base_totale.groupby('state').agg({
    'n_injured': 'sum',
    'n_killed': 'sum',
    'resident_pop_year_2015':'sum'
}).reset_index()
grouped_data7['injured_per_1K']=grouped_data7["n_injured"]/grouped_data7['resident_pop_year_2015']*100
grouped_data7['killed_per_1K']=grouped_data7["n_killed"]/grouped_data7['resident_pop_year_2015']*100

fig7 = px.bar(
    grouped_data7,
    x=grouped_data7['state'],
    y=['injured_per_1K', 'killed_per_1K'],
    labels={"state": "Etat", "value": "nombre de victimes pour 1000 par incident", "variable": "Type de victime"},
    title="Nombre moyen de victimes d'incident par arme à feu par Etat",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig7.update_layout(barmode='group')  # Display bars side by side

fig7.show()#truc chelou pour le nombre 

In [83]:
grouped_data7 = base_totale.groupby('state').agg({
    'n_injured': 'sum',
    'n_killed': 'sum',
    'resident_pop_year_2015':'sum'
}).reset_index()
grouped_data7['injured_per_1K']=grouped_data7["n_injured"]/grouped_data7['resident_pop_year_2015']
grouped_data7['killed_per_1K']=grouped_data7["n_killed"]/grouped_data7['resident_pop_year_2015']
grouped_data7

,state,n_injured,n_killed,resident_pop_year_2015,injured_per_1K,killed_per_1K
0,Alabama,2952,1821,1.768880e+06,0.001669,0.001029
1,Arizona,1060,1063,6.807424e+06,0.000156,0.000156
2,Arkansas,1302,735,5.693581e+05,0.002287,0.001291
3,California,7504,5397,4.944978e+07,0.000152,0.000109
4,Colorado,1088,742,1.460468e+06,0.000745,0.000508
5,Connecticut,1221,326,2.475435e+06,0.000493,0.000132
6,Delaware,838,209,7.328678e+05,0.001143,0.000285
7,District of Columbia,1377,430,2.098748e+06,0.000656,0.000205
8,Florida,6858,3734,1.448326e+07,0.000474,0.000258
9,Georgia,3964,2348,2.908065e+06,0.001363,0.000807


In [79]:
grouped_data5

,state,n_injured,n_killed
0,Alabama,2952,1821
1,Arizona,1060,1063
2,Arkansas,1302,735
3,California,7504,5397
4,Colorado,1088,742
5,Connecticut,1221,326
6,Delaware,838,209
7,District of Columbia,1377,430
8,Florida,6858,3734
9,Georgia,3964,2348
